In [ ]:
db.inventory.insertMany([
   { item: "journal", qty: 25, tags: ["blank", "red"], size: { h: 14, w: 21, uom: "cm" } },
   { item: "mat", qty: 85, tags: ["gray"], size: { h: 27.9, w: 35.5, uom: "cm" } },
   { item: "mousepad", qty: 25, tags: ["gel", "blue"], size: { h: 19, w: 22.85, uom: "cm" } }
])

In [ ]:
#In MongoDB, each document(ROW) stored in a collection(TABLE) requires a unique _id field that acts as a primary key.
#If an inserted document omits the _id field, 
#the MongoDB driver automatically generates an ObjectId for the _id field.

#SELECT * FROM inventory
db.inventory.find( {} )

In [ ]:
#Specify Equality Condition
#SELECT * FROM inventory WHERE status = "D"
db.inventory.find( { status: "D" } )

In [ ]:
#SELECT * FROM inventory WHERE status in ("A", "D")
db.inventory.find( { status: { $in: [ "A", "D" ] } } )

In [ ]:
#SELECT * FROM inventory WHERE status = "A" AND qty < 30
db.inventory.find( { status: "A", qty: { $lt: 30 } } )

In [ ]:
#SELECT * FROM inventory WHERE status = "A" OR qty < 30
db.inventory.find( { $or: [ { status: "A" }, { qty: { $lt: 30 } } ] } )

In [ ]:
#SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")
db.inventory.find( {
     status: "A",
     $or: [ { qty: { $lt: 30 } }, { item: /^p/ } ]
} )

In [ ]:
#Match an Nested Document
db.inventory.find( { size: { h: 14, w: 21, uom: "cm" } } )
# the following query does not match any documents in the inventory collection:order 
#the field(column) size
db.inventory.find(  { size: { w: 21, h: 14, uom: "cm" } }  )

In [ ]:
# selects all documents where the field uom nested in the size field equals "in"
db.inventory.find( { "size.uom": "in" } )

In [ ]:
db.inventory.find( { "size.h": { $lt: 15 }, "size.uom": "in", status: "D" } )

In [ ]:
#Query an Array 
db.inventory.insertMany([
   { item: "journal", qty: 25, tags: ["blank", "red"], dim_cm: [ 14, 21 ] },
   { item: "notebook", qty: 50, tags: ["red", "blank"], dim_cm: [ 14, 21 ] },
   { item: "paper", qty: 100, tags: ["red", "blank", "plain"], dim_cm: [ 14, 21 ] },
   { item: "planner", qty: 75, tags: ["blank", "red"], dim_cm: [ 22.85, 30 ] },
   { item: "postcard", qty: 45, tags: ["blue"], dim_cm: [ 10, 15.25 ] }
]);

In [ ]:
#specified order
db.inventory.find( { tags: ["red", "blank"] } )
# without regard to order or other elements 
db.inventory.find( { tags: { $all: ["red", "blank"] } } )

In [ ]:
#queries for all documents where the array dim_cm contains at least one element 
#whose value is greater than 25
db.inventory.find( { dim_cm: { $gt: 25 } } )


In [ ]:
#Multiple Conditions for Array Elements
#at least one element that is both greater than ($gt) 22 and less than ($lt) 30
db.inventory.find( { dim_cm: { $elemMatch: { $gt: 22, $lt: 30 } } } )

In [ ]:
#the second element in the array dim_cm is greater than 25
db.inventory.find( { "dim_cm.1": { $gt: 25 } } )

In [ ]:
#selects documents where the array tags has 3 elements.
db.inventory.find( { "tags": { $size: 3 } } )

In [ ]:
db.inventory.insertMany( [
   { item: "journal", instock: [ { warehouse: "A", qty: 5 }, { warehouse: "C", qty: 15 } ] },
   { item: "notebook", instock: [ { warehouse: "C", qty: 5 } ] },
   { item: "paper", instock: [ { warehouse: "A", qty: 60 }, { warehouse: "B", qty: 15 } ] },
   { item: "planner", instock: [ { warehouse: "A", qty: 40 }, { warehouse: "B", qty: 5 } ] },
   { item: "postcard", instock: [ { warehouse: "B", qty: 15 }, { warehouse: "C", qty: 35 } ] }
]);

In [ ]:
#Query for a Document Nested in an Array
db.inventory.find( { "instock": { warehouse: "A", qty: 5 } } )
#Equality matches on the whole embedded/nested document require 
#an exact match of the specified document including the field order
db.inventory.find( { "instock": { qty: 5, warehouse: "A" } } )
#the same results like the first query
db.inventory.find( { "instock": { $elemMatch: { qty: 5, warehouse: "A" } } } )

In [ ]:
db.inventory.find( { "instock.qty": 5, "instock.warehouse": "A" } )
#results:
#   { item: "journal", instock: [ { warehouse: "A", qty: 5 }, { warehouse: "C", qty: 15 } ] },
#   { item: "planner", instock: [ { warehouse: "A", qty: 40 }, { warehouse: "B", qty: 5 } ] },


In [ ]:
#Return the Specified Fields and the _id 
#SELECT _id, item, status from inventory WHERE status = "A"
db.inventory.find( { status: "A" }, { item: 1, status: 1 } )
#SELECT item, status from inventory WHERE status = "A"
db.inventory.find( { status: "A" }, { item: 1, status: 1,_id:0 } )


In [ ]:
#Return All But the Excluded Fields
db.inventory.find( { status: "A" }, { status: 0, instock: 0 } )


In [ ]:
#Return Specific Fields in Embedded Documents
db.inventory.find(
   { status: "A" },
   { item: 1, status: 1, "size.uom": 1 }
)
#All other fields are returned in the matching documents
db.inventory.find(
   { status: "A" },
   { "size.uom": 0 }
)


In [ ]:
#Projection on Embedded Documents in an Array
db.inventory.find( { status: "A" }, { item: 1, status: 1, "instock.qty": 1 } )
# $slice- $slice projection operator to return the last element in the instock array:
db.inventory.find( { status: "A" }, { item: 1, status: 1, instock: { $slice: -1 } } )


In [ ]:
#Query for Null or Missing Fields

In [ ]:
#The { item : null } query matches documents that either contain the item field 
#whose value is null or that do not
#contain the item field.

db.inventory.insertMany([
   { _id: 1, item: null },
   { _id: 2 }
])
#Equality Filter
#{ _id: 1, item: null },
#{ _id: 2 }
db.inventory.find( { item: null } )

#Type Check
#{ _id: 1, item: null }
db.inventory.find( { item : { $type: 10 } } )

#Existence Check
#The { item : { $exists: false } } query matches documents that do not contain 
#the item field:
#{ _id: 2 }
db.inventory.find( { item : { $exists: false } } )




In [ ]:
#update

In [ ]:
db.inventory.insertMany( [
   { item: "canvas", qty: 100, size: { h: 28, w: 35.5, uom: "cm" }, status: "A" },
   { item: "journal", qty: 25, size: { h: 14, w: 21, uom: "cm" }, status: "A" },
   { item: "mat", qty: 85, size: { h: 27.9, w: 35.5, uom: "cm" }, status: "A" },
   { item: "mousepad", qty: 25, size: { h: 19, w: 22.85, uom: "cm" }, status: "P" },
   { item: "notebook", qty: 50, size: { h: 8.5, w: 11, uom: "in" }, status: "P" },
   { item: "paper", qty: 100, size: { h: 8.5, w: 11, uom: "in" }, status: "D" },
   { item: "planner", qty: 75, size: { h: 22.85, w: 30, uom: "cm" }, status: "D" },
   { item: "postcard", qty: 45, size: { h: 10, w: 15.25, uom: "cm" }, status: "A" },
   { item: "sketchbook", qty: 80, size: { h: 14, w: 21, uom: "cm" }, status: "A" },
   { item: "sketch pad", qty: 95, size: { h: 22.85, w: 30.5, uom: "cm" }, status: "A" }
] );

In [ ]:
#update the first document where item equals "paper"
#Update a Single Document
db.inventory.updateOne(
   { item: "paper" },
   {
     $set: { "size.uom": "cm", status: "P" },
     $currentDate: { lastModified: true }
   }
)
#Update Multiple Documents
db.inventory.updateMany(
   { "qty": { $lt: 50 } },
   {
     $set: { "size.uom": "in", status: "P" },
     $currentDate: { lastModified: true }
   }
)
#Replace a Document
db.inventory.replaceOne(
   { item: "paper" },
   { item: "paper", instock: [ { warehouse: "A", qty: 60 }, { warehouse: "B", qty: 40 } ] }
)

In [ ]:
db.students3.insert([
   { "_id" : 1, "tests" : [ 95, 92, 90 ], "modified" : ISODate("2019-01-01T00:00:00Z") },
   { "_id" : 2, "tests" : [ 94, 88, 90 ], "modified" : ISODate("2019-01-01T00:00:00Z") },
   { "_id" : 3, "tests" : [ 70, 75, 82 ], "modified" : ISODate("2019-01-01T00:00:00Z") }
]);

In [ ]:
db.students3.updateMany(
   { },
   [
     { $set: { average : { $trunc: [ { $avg: "$tests" }, 0 ] }, modified: "$$NOW" } },
     { $set: { grade: { $switch: {
                           branches: [
                               { case: { $gte: [ "$average", 90 ] }, then: "A" },
                               { case: { $gte: [ "$average", 80 ] }, then: "B" },
                               { case: { $gte: [ "$average", 70 ] }, then: "C" },
                               { case: { $gte: [ "$average", 60 ] }, then: "D" }
                           ],
                           default: "F"
     } } } }
   ]
)

In [ ]:

db.orders.insertMany([
   { _id: 1, cust_id: "Ant O. Knee", ord_date: new Date("2020-03-01"), price: 25, items: [ { sku: "oranges", qty: 5, price: 2.5 }, { sku: "apples", qty: 5, price: 2.5 } ], status: "A" },
   { _id: 2, cust_id: "Ant O. Knee", ord_date: new Date("2020-03-08"), price: 70, items: [ { sku: "oranges", qty: 8, price: 2.5 }, { sku: "chocolates", qty: 5, price: 10 } ], status: "A" },
   { _id: 3, cust_id: "Busby Bee", ord_date: new Date("2020-03-08"), price: 50, items: [ { sku: "oranges", qty: 10, price: 2.5 }, { sku: "pears", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 4, cust_id: "Busby Bee", ord_date: new Date("2020-03-18"), price: 25, items: [ { sku: "oranges", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 5, cust_id: "Busby Bee", ord_date: new Date("2020-03-19"), price: 50, items: [ { sku: "chocolates", qty: 5, price: 10 } ], status: "A"},
   { _id: 6, cust_id: "Cam Elot", ord_date: new Date("2020-03-19"), price: 35, items: [ { sku: "carrots", qty: 10, price: 1.0 }, { sku: "apples", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 7, cust_id: "Cam Elot", ord_date: new Date("2020-03-20"), price: 25, items: [ { sku: "oranges", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 8, cust_id: "Don Quis", ord_date: new Date("2020-03-20"), price: 75, items: [ { sku: "chocolates", qty: 5, price: 10 }, { sku: "apples", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 9, cust_id: "Don Quis", ord_date: new Date("2020-03-20"), price: 55, items: [ { sku: "carrots", qty: 5, price: 1.0 }, { sku: "apples", qty: 10, price: 2.5 }, { sku: "oranges", qty: 10, price: 2.5 } ], status: "A" },
   { _id: 10, cust_id: "Don Quis", ord_date: new Date("2020-03-23"), price: 25, items: [ { sku: "oranges", qty: 10, price: 2.5 } ], status: "A" }
])

In [ ]:
#MapReduce
var mapFunction1 = function() {
   emit(this.cust_id, this.price);
};

In [ ]:
var reduceFunction1 = function(keyCustId, valuesPrices) {
   return Array.sum(valuesPrices);
};

In [ ]:
db.orders.mapReduce(
   mapFunction1,
   reduceFunction1,
   { out: "map_reduce_example" }
)